In [1]:
import time

from bokeh.plotting import figure, show, Session, cursession, output_server
from bokeh.models import DatetimeTickFormatter

from data.datahandler import yield_from_last
from data.datastream import queryinput_to_csv_yf, split_csvline_yf, queryinput_to_gen_yf

In [2]:
session = Session(load_from_config=False, root_url = "http://127.0.0.1:5006/")
output_server("candlestick", url = "http://127.0.0.1:5006/")
#session.register("vabite","clanugu")
#session.login("vabite", "clanugu")

Using saved session configuration for http://127.0.0.1:5006/
To override, pass 'load_from_config=False' to Session


In [3]:
class Candlestick(object):
    
    #le variabili del plot non inizializzate (titolo, etc.) sono comunque accassibili dall'esterno
    def __init__(self, candle_visualized_n = 15, quadwidth_percentual = 0.5, tmstep = 1): #step tra due dati in gg
        #inizializza dati per plot
        #tmstep rappresenta la distanza temporale attesa tra due dati.
        #Con tale passo temporale è shiftato l'asse x in caso assenza dati (modellato con arrivo None), definita
        #l'ampiezza dell'asse x mostrata, e scalata l'ampiezza x dei glifi. E' convertita in in ms, udm usata da Bokeh
        #Possibile anche ricavarlo dinamicamente da dati ricevuti se si suppone intervallo tra due dati possa variare
        self.tmstep = tmstep * 24 * 60 * 60 * 1000
        #quadwidth ampiezza dei rettangoli reale. quadwidth_percentual ampiezza dei retangoli relativa al tmstep
        self.quadwidth = quadwidth_percentual * self.tmstep 
        self.candle_visualized_n = candle_visualized_n #numero di candele da visualizzare sul grafico
        #inizializza plot
        self.plot = figure(plot_width = 900, plot_height = 300)
        self.plot.xaxis[0].formatter = DatetimeTickFormatter(
            formats = dict(
                hours = ["d %b %Y"], #numero giorno, sigla mese, ora come intero da 00 a 23
                days = ["%d %b %Y"], #numero giorno, sigla mese, numero anno
                months = ["%d %b %Y"], #numero giorno, sigla mese, numero anno
                years = ["%d %b %Y"]) #numero giorno, sigla mese, numero anno
            )
        #grafico inizializzato vuoto inizializzando tutti i data source come liste vuote
        self.plot.quad(top = [], bottom = [], left = [], right = [], fill_color = [], name = "quads") 
        self.plot.segment(x0 = [], x1 = [], y0 = [], y1 = [], name = "segments")
        #estremi dell'instervallo in x mostrato inizializzati dal costruttore in quanto, se il primo dato è None, 
        #in update non riesce ad inizializzarli dovendo sommare un intero ad un None. Di default inizializzati 
        #all'istante attuale (se si fosse inizializzato a 0 sarebbero stati inizializzati alla epoch; tuttavia ho 
        #dal test pare non cambi molto, non visualizzando lui il range degli assi fino a che non visualizza un glifo)
        self.plot.x_range.start = time.time() * 1000 
        self.plot.x_range.end = time.time() * 1000
        self.ds_quads = self.plot.select({"name":"quads"})[0].data_source
        self.ds_segments = self.plot.select({"name":"segments"})[0].data_source

    #ritorna l'oggetto Plot
    def get_plot(self):
        return self.plot        

    #IN:lista (wrap effettuato per rendere la sintassi del metodo uguale per i vari chart) in cui ogni elemento è:
    #A) una linea del csv risultante da una query a YF, se dato valido
    #B) None, se simulato arrivo dato invalido o assenza dati
    #Nel caso specifico la lista contiene un solo elemento relativo all'unico titolo della query
    #OUT: lista dei valori da passare al metodo update() per aggiornare il grafico
    #A) [data in millisecondi a partire da epoch, apertura, massimo, minimo, chiusura]; se dato valido
    #B) None, se simulato arrivo dato invalido o assenza dati
    def csvlines_to_data_yf(self, csvlines):
        if csvlines[0] == None: data = [None]
        else: data = split_csvline_yf(csvlines[0], "int")[0:5]
        return data
    
    #aggiorna i ds e ritorna una lista degli oggetti di cui fare push a partire da una linea csv formattata come da YF
    def update(self, data):
        #recupera una nuova lista di dati che suppone strutturata come segue:
        #indice 0: data in millisecondi a partire da epoch
        #indice 1: apertura
        #indice 2: massimo
        #indice 3: minimo
        #indice 4: chiusura
        #L'arrivo di dati non validi o l'assenza di dati è simulata come arrivo di None
        if data != [None]: #caso arrivo valido
            #aggiorna dati quad
            self.ds_quads.data["top"].append(data[4])
            self.ds_quads.data["bottom"].append(data[1])
            self.ds_quads.data["left"].append(data[0] - self.quadwidth / 2)
            self.ds_quads.data["right"].append(data[0] + self.quadwidth / 2)
            if data[4] > data[1]: self.ds_quads.data["fill_color"].append("green")
            else: self.ds_quads.data["fill_color"].append("red")
            #aggiorna dati segment
            self.ds_segments.data["x0"].append(data[0])
            self.ds_segments.data["x1"].append(data[0])
            self.ds_segments.data["y0"].append(data[3])
            self.ds_segments.data["y1"].append(data[2])
            #aggiorna i dati x_range di plot
            #in caso di arrivo dato valido, definisce un intervallo di visualizzazione ponendo la candela del dato in
            #arrivo a mezzo tmstep dal bordo dx del grafico e a candle_visualized_n + 0.5 tmstep dal bordo sx
            self.plot.x_range.start = data[0] - (self.candle_visualized_n - 0.5) * self.tmstep
            self.plot.x_range.end = data[0] + 0.5 * self.tmstep
        else: #caso ricezione dato non valido
            #shifta di un tmstep a sx l'intervallo di visualizzazione nell'asse x e non aggiunge candele al grafico
            #NB: mentre in caso arrivo dati validi stabilisce il range di visualizzazione in base all'ascissa del dato
            #arrivato, qui effettua lo shift con tmstep.Se tmstep non è preso esattamente pari alla distanza temporale
            #tra l'arrivo di due dati:
            #arrivo dati validi: mostra un intervallo in x non esattamente uguale a quello desiderato
            #arrivo dati non validi o asenza dati: qui accumula ritardo o anticipo rispetto ai dati attuali
            self.plot.x_range.start += self.tmstep 
            self.plot.x_range.end += self.tmstep
        #ritorna una lista degli oggetti di cui fare push: data sources dei glifi e x_range di plot
        return [self.ds_quads, self.ds_segments, self.plot.x_range]

In [4]:
#inizializzazione variabili
#parametri utilizzati per effettuare la query a Yahoo Finance
quote_id = "UCG.MI"
start_y = 2013
start_m = 1
start_d = 1
stop_y = 2014
stop_m = 12
stop_d = 31
step = "d" 

#parametri relativi al grafico
tmdict = {"d":1, "w":7, "m":(365 * 4 + 1) / (12 * 4)}
candle_visualized_n = 15
quadwidth_percentual = 0.2
tmstep = tmdict[step] #per i mesi preso step pari ai giorni in 4 anni diviso i mesi in 4 anni

In [5]:
datagen = queryinput_to_gen_yf(quote_id, start_y, start_m, start_d, stop_y, stop_m, stop_d, step = "d")
c = Candlestick(candle_visualized_n, quadwidth_percentual, tmstep)
show(c.get_plot())
while True:
    data = c.csvlines_to_data_yf([next(datagen)])##; print(data)
    cursession().store_objects(*c.update(data))
    time.sleep(0.5)

KeyboardInterrupt: 

In [ ]:
#TEST CLASSE CandlestickYF

In [ ]:
#simulo dati validi formattati come csv di YF
#validline1 differiscono solo per data 
validline1 = b'2014-01-03,1.000,2.000,3.000,4.000,1000000,5.000\n'
    #delta temporale giornaliero
validline1A = b'2014-01-04,1.000,2.000,3.000,4.000,1000000,5.000\n'
validline1B = b'2014-01-05,1.000,2.000,3.000,4.000,1000000,5.000\n'
    #delta temporale mensile
validline1C = b'2014-02-03,1.000,2.000,3.000,4.000,1000000,5.000\n'
validline1D = b'2014-03-03,1.000,2.000,3.000,4.000,1000000,5.000\n'
    #delta temporale annuale
validline1E = b'2015-01-03,1.000,2.000,3.000,4.000,1000000,5.000\n'
validline1F = b'2016-01-03,1.000,2.000,3.000,4.000,1000000,5.000\n'
#validline2 per data e valore
validline2 = b'2014-01-01,3.000,4.000,5.000,6.000,2000000,7.000\n'
validline2A = b'2014-01-04,5.000,4.000,3.000,2.000,3000000,1.000\n'
validline2B = b'2014-01-15,8.000,11.000,14.000, 16.000,1000000,18.000\n'
validline2C = b'2014-01-30,25.000,20.000,15.000,10.000,500000,5.000\n'
validline2D = b'2014-01-31,-1,-2,-3,-4,500000,-5\n'
invalidline = None

In [ ]:
#testa arrivo dato non valido o assenza dati subito
c = Candlestick()
show(c.get_plot())
cursession().store_objects(*c.update(c.csvlines_to_data_yf([invalidline])))

In [ ]:
#testa arrivo dato non valido o assenza di dati, seguita da dato valido
c = Candlestick()
show(c.get_plot())
cursession().store_objects(*c.update(c.csvlines_to_data_yf([invalidline])))
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline1])))

In [ ]:
#testa arrivo dati validi con stessa data
c = Candlestick()
show(c.get_plot())
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline1])))
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline1])))

In [ ]:
#testa arrivo dati validi con step temporale 1gg e arrivo dato non valido con tmstep definito da utente uguale a 1gg
c = Candlestick(tmstep = 1)
show(c.get_plot())
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline1]))) #dato centrato 4 Jan
time.sleep(3)
cursession().store_objects(*c.update(c.csvlines_to_data_yf([invalidline]))) #shift asse x di 1gg
time.sleep(3)
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline1B]))) #dato centrato 6 Jan

In [ ]:
#testa arrivo dati validi con step temporale 1m e arrivo dato non valido con tmstep definito da utente uguale a 1m
c = Candlestick(tmstep = 30)
show(c.get_plot())
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline1]))) #dato centrato Jan
time.sleep(3)
cursession().store_objects(*c.update(c.csvlines_to_data_yf([invalidline]))) #shift asse x di 30gg
time.sleep(3)
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline1D]))) #dato centrato Mar

In [ ]:
#testa arrivo dati validi con step temporale annuale
c = Candlestick(tmstep = 365)
show(c.get_plot())
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline1]))) #dato centrato 2014
time.sleep(3)
cursession().store_objects(*c.update(c.csvlines_to_data_yf([invalidline]))) #shift asse x di 365gg
time.sleep(3)
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline1E]))) #dato centrato 2015

In [ ]:
#testa arrivo dato valido con close-open>0 seguito da dato valido con close-open<0
c = Candlestick(tmstep = 1)
show(c.get_plot())
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline2]))) #dato centrato 1 Jan
time.sleep(3)
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline2A]))) #dato centrato 4 Jan

In [ ]:
#testa larghezza intervallo visualizzazione x di default
c = Candlestick()
show(c.get_plot())
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline2]))) #dato centrato 1 Jan
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline2B]))) #dato centrato 15 Jan

In [ ]:
#testa larghezza intervallo visualizzazione x fornito da costruttore
c = Candlestick(candle_visualized_n = 30)
show(c.get_plot())
#dato centrato 1 Jan. Dato a estremo sx grafico
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline2]))) 
#dato centrato 30 Jan. Dato a estremo dx grafico
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline2C])))

In [ ]:
#testa larghezza candele definita da utente
c = Candlestick(quadwidth_percentual = 0.95) #i due dati successivi devono quasi toccarsi
show(c.get_plot())
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline1A]))) 
cursession().store_objects(*c.update(c.csvlines_to_data_yf([validline1B]))) 